# 향후 판매량 예측 경진대회 모델링
- [향후 판매량 예측 경진대회 링크](https://www.kaggle.com/c/competitive-data-science-predict-future-sales)
- [모델링 코드 참고 링크](https://www.kaggle.com/dkomyagin/predict-future-sales-lightgbm-framework)

In [1]:
# ch6/predict_future_sales_modeling.ipynb

import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore') # 경고 메시지 생략

# 데이터 경로
data_path = '/kaggle/input/competitive-data-science-predict-future-sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

#### 피처명 한글로 변경

In [2]:
sales_train = sales_train.rename(columns={'date': '날짜', 
                                          'date_block_num': '날짜ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품범주ID'})

item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품범주명',
                                          'item_category_id': '상품범주ID'})

test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

#### 데이터 형변환

In [3]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

62.5% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


## 피처 엔지니어링 I - sales_train, shops, items, item_categories 처리

### sales_train 이상치 제거 및 전처리

In [4]:
# 판매가가 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매가'] > 0]
# 판매가가 50,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매가'] < 50000]
# 판매량이 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매량'] > 0]
# 판매량이 1,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매량'] < 1000]

In [5]:
print(shops['상점명'][0], '||', shops['상점명'][57])
print(shops['상점명'][1], '||', shops['상점명'][58])
print(shops['상점명'][10], '||', shops['상점명'][11])
print(shops['상점명'][39], '||', shops['상점명'][40])

!Якутск Орджоникидзе, 56 фран || Якутск Орджоникидзе, 56
!Якутск ТЦ "Центральный" фран || Якутск ТЦ "Центральный"
Жуковский ул. Чкалова 39м? || Жуковский ул. Чкалова 39м²
РостовНаДону ТРК "Мегацентр Горизонт" || РостовНаДону ТРК "Мегацентр Горизонт" Островной


In [6]:
# sales_train 데이터에서 상점ID 수정
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

# test 데이터에서 상점ID 수정
test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

### shops 파생 피처 생성 및 인코딩

In [7]:
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])

In [8]:
shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [9]:
shops.loc[shops['도시'] =='!Якутск', '도시'] = 'Якутск'

In [10]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()
# 도시 피처 레이블 인코딩
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [11]:
# 상점명 피처 제거
shops = shops.drop('상점명', axis=1)

shops.head()

,상점ID,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


### itmes 파생 피처 생성

In [12]:
# 상품명 피처 제거
items = items.drop(['상품명'], axis=1)

In [13]:
# 상품이 맨 처음 팔린 날을 피처로 추가
items['첫 판매날짜'] = sales_train.groupby('상품ID').agg({'날짜ID': 'min'})['날짜ID']

items.head()

,상품ID,상품범주ID,첫 판매날짜
0,0,40,20.0
1,1,76,15.0
2,2,40,19.0
3,3,40,18.0
4,4,40,20.0


In [14]:
items[items['첫 판매날짜'].isna()]

,상품ID,상품범주ID,첫 판매날짜
83,83,40,NaN
140,140,45,NaN
168,168,44,NaN
173,173,45,NaN
204,204,44,NaN
...,...,...,...
21974,21974,61,NaN
21975,21975,61,NaN
22022,22022,40,NaN
22035,22035,40,NaN


In [15]:
# 첫 판매날짜 피처의 결측값을 34로 대체
items['첫 판매날짜'] = items['첫 판매날짜'].fillna(34)

### item_categories 파생 피처 생성 및 인코딩

In [16]:
# 상품범주명의 첫 단어를 범주분류로 추출
item_categories['범주분류'] = item_categories['상품범주명'].apply(lambda x: x.split()[0])  

In [17]:
item_categories['범주분류'].value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Программы      6
Музыка         6
Кино           5
Карты          5
Чистые         2
Служебные      2
Доставка       1
Билеты         1
Элементы       1
PC             1
Name: 범주분류, dtype: int64

In [18]:
def make_etc(x):
    if len(item_categories[item_categories['범주분류']==x]) >= 5:
        return x
    else:
        return 'etc'

# 범주분류 개수가 5개 미만이면 'etc'로 바꾸기
item_categories['범주분류'] = item_categories['범주분류'].apply(make_etc)

In [19]:
item_categories.head()

,상품범주명,상품범주ID,범주분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


In [20]:
# 레이블 인코더 생성
label_encoder = LabelEncoder()
# 범주분류 피처 레이블 인코딩
item_categories['범주분류'] = label_encoder.fit_transform(item_categories['범주분류'])

# 상품범주명 피처 제거
item_categories = item_categories.drop('상품범주명', axis=1)

### 데이터 조합 및 파생 피처 생성

In [21]:
from itertools import product

train = []
# 날짜ID, 상점ID, 상품ID 조합 생성
for i in sales_train['날짜ID'].unique():
    all_shop = sales_train.loc[sales_train['날짜ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['날짜ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['날짜ID', '상점ID', '상품ID'] # 기준 피처 # 기준 피처
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [22]:
group = sales_train.groupby(idx_features).agg({'판매량': 'sum',
                                               '판매가': 'mean'})
group = group.reset_index()
group = group.rename(columns={'판매량': '총 판매량', '판매가': '평균 판매가'})

train = train.merge(group, on=idx_features, how='left')

train.head()

,날짜ID,상점ID,상품ID,총 판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


In [23]:
import gc

# group 변수 가비지 컬렉션
del group
gc.collect();

In [24]:
# 상품 판매건수 피처 추가
group = sales_train.groupby(idx_features).agg({'판매량': 'count'})
group = group.reset_index()
group = group.rename(columns={'판매량': '판매건수'})

train = train.merge(group, on=idx_features, how='left')

# 가비지 컬렉션
del group, sales_train
gc.collect()

train.head()

,날짜ID,상점ID,상품ID,총 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


### 테스트 데이터 이어붙이기, 나머지 데이터 병합

In [25]:
# 테스트 데이터 날짜ID를 34로 설정
test['날짜ID'] = 34

# train과 test 이어붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)
# 결측값을 0으로 대체
all_data = all_data.fillna(0)

all_data.head()

,날짜ID,상점ID,상품ID,총 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0


In [26]:
# 나머지 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품범주ID', how='left')

# 데이터 형변환
all_data = downcast(all_data)

# 가비지 컬렉션
del shops, items, item_categories
gc.collect();

64.6% 압축됨


## 피처 엔지니어링 II - 시차 피처 생성

### 기준 피처별 월간 평균 판매량 파생 피처 생성

In [27]:
def add_mean_features(df, mean_features, idx_features):
    # 기준 피처 확인 ---①
    assert (idx_features[0] == '날짜ID') and \
           len(idx_features) in [2, 3]
    
    # 파생 피처명 설정 ---②
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'
    
    # 기준 피처를 토대로 그룹화해 월간 평균 판매량 구하기 ---③
    group = df.groupby(idx_features).agg({'총 판매량': 'mean'})
    group = group.reset_index()
    group = group.rename(columns={'총 판매량': feature_name})
    
    # df와 group 병합 ---④
    df = df.merge(group, on=idx_features, how='left')
    # 데이터 형변환 ---⑤
    df = downcast(df, False)
    # 새로 만든 mean_feature_name 피처를 mean_features 리스트에 추가 ---⑥
    mean_features.append(feature_name)
    
    # 가비지 컬렉션
    del group
    gc.collect()
    
    return df, mean_features

In [28]:
# 그룹화 기준 피처에 '상품ID'를 포함한 파생 피처를 담을 리스트
item_mean_features = []

# ['날짜ID', '상품ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['날짜ID', '상품ID'])

# ['날짜ID', '상품ID', '도시']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['날짜ID', '상품ID', '도시'])

In [29]:
item_mean_features

['상품ID별 평균 판매량', '상품ID 도시별 평균 판매량']

In [30]:
# 그룹화 기준 피처에 '상점ID'를 포함한 파생 피처를 담을 리스트
shop_mean_features = []

# ['날짜ID', '상점ID', '상품범주ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, shop_mean_features = add_mean_features(df=all_data, 
                                                 mean_features=shop_mean_features,
                                                 idx_features=['날짜ID', '상점ID', '상품범주ID'])

In [31]:
shop_mean_features

['상점ID 상품범주ID별 평균 판매량']

### 시차 파생 피처 만들기

In [32]:
def add_lag_features(df, lag_features_to_clip, idx_features, 
                     lag_feature, nlags=3, clip=False):
    # 시차 피처 생성에 필요한 DataFrame 부분만 복사 ---①
    df_temp = df[idx_features + [lag_feature]].copy() 

    # 시차 피처 생성 ---②
    for i in range(1, nlags+1):
        # 시차 피처명 ---③
        lag_feature_name = lag_feature +'_시차' + str(i)
        # df_temp 열 이름 설정 ---④
        df_temp.columns = idx_features + [lag_feature_name]
        # df_temp의 date_block_num 피처에 1 더하기 ---⑤
        df_temp['날짜ID'] += i
        # idx_feature를 기준으로 df와 df_temp 병합하기 ---⑥
        df = df.merge(df_temp.drop_duplicates(), 
                      on=idx_features, 
                      how='left')
        # 결측값 0으로 대체 ---⑦
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        # 0~20 사이로 제한할 시차 피처를 lag_features_to_clip에 추가 ---⑧
        if clip: 
            lag_features_to_clip.append(lag_feature_name)
    
    # 데이터 형변환
    df = downcast(df, False)
    # 가비지 컬렉션
    del df_temp
    gc.collect()
    
    return df, lag_features_to_clip

In [33]:
lag_features_to_clip = [] # 0~20 사이로 제한할 시차 피처를 담을 리스트
idx_features = ['날짜ID', '상점ID', '상품ID'] # 기준 피처

# idx_features를 기준으로 총 판매량의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='총 판매량', 
                                                  nlags=3,
                                                  clip=True)

In [34]:
all_data.head().T

,0,1,2,3,4
날짜ID,0.000000,0.000000,0.000000,0.000000,0.000000
상점ID,59.000000,59.000000,59.000000,59.000000,59.000000
상품ID,22154.000000,2552.000000,2554.000000,2555.000000,2564.000000
총 판매량,1.000000,0.000000,0.000000,0.000000,0.000000
평균 판매가,999.000000,0.000000,0.000000,0.000000,0.000000
판매건수,1.000000,0.000000,0.000000,0.000000,0.000000
도시,30.000000,30.000000,30.000000,30.000000,30.000000
상품범주ID,37.000000,58.000000,58.000000,56.000000,59.000000
첫 판매날짜,0.000000,0.000000,0.000000,0.000000,0.000000
범주분류,5.000000,7.000000,7.000000,7.000000,7.000000


In [35]:
lag_features_to_clip

['총 판매량_시차1', '총 판매량_시차2', '총 판매량_시차3']

In [36]:
# idx_features를 기준으로 판매건수 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수', 
                                                  nlags=3)

# idx_features를 기준으로 평균 판매가 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가', 
                                                  nlags=3)

In [37]:
X_test_temp = all_data[all_data['날짜ID'] == 34]
X_test_temp[item_mean_features].sum()

상품ID별 평균 판매량       0.0
상품ID 도시별 평균 판매량    0.0
dtype: float32

In [38]:
# idx_features를 기준으로 item_mean_features 요소별 시차 피처 생성
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=idx_features, 
                                                      lag_feature=item_mean_feature, 
                                                      nlags=3)
# item_mean_features 피처 제거
all_data = all_data.drop(item_mean_features, axis=1)

In [39]:
shop_mean_features

['상점ID 상품범주ID별 평균 판매량']

In [40]:
# ['날짜ID', '상점ID', '상품범주ID']를 기준으로 shop_mean_features 요소별 시차 피처 생성
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=['날짜ID', '상점ID', '상품범주ID'], 
                                                      lag_feature=shop_mean_feature, 
                                                      nlags=3)
# shop_mean_features 피처 제거
all_data = all_data.drop(shop_mean_features, axis=1)

In [41]:
# 날짜 ID 3미만인 데이터 제거
all_data = all_data.drop(all_data[all_data['날짜ID'] < 3].index)

### 기타 피처 엔지니어링

In [42]:
all_data['총 판매량 시차평균'] = all_data[['총 판매량_시차1',
                                         '총 판매량_시차2', 
                                         '총 판매량_시차3']].mean(axis=1)

In [43]:
# 0~20 사이로 값 제한
all_data[lag_features_to_clip + ['총 판매량', '총 판매량 시차평균']] = all_data[lag_features_to_clip +['총 판매량', '총 판매량 시차평균']].clip(0, 20)

In [44]:
all_data['시차변화량1'] = all_data['총 판매량_시차1']/all_data['총 판매량_시차2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

all_data['시차변화량2'] = all_data['총 판매량_시차2']/all_data['총 판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

In [45]:
all_data['신상여부'] = all_data['첫 판매날짜'] == all_data['날짜ID']

In [46]:
all_data['첫 판매 후 기간'] = all_data['날짜ID'] - all_data['첫 판매날짜']

In [47]:
all_data['월'] = all_data['날짜ID']%12

In [48]:
# 첫 판매날짜, 평균 판매가, 판매건수 피처 제거
all_data = all_data.drop(['첫 판매날짜', '평균 판매가', '판매건수'], axis=1)
all_data = downcast(all_data, False) # 데이터 형변환
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9904582 entries, 1122386 to 11026967
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   날짜ID                     int8   
 1   상점ID                     int8   
 2   상품ID                     int16  
 3   총 판매량                    int8   
 4   도시                       int8   
 5   상품범주ID                   int8   
 6   범주분류                     int8   
 7   총 판매량_시차1                int8   
 8   총 판매량_시차2                int8   
 9   총 판매량_시차3                int8   
 10  판매건수_시차1                 int8   
 11  판매건수_시차2                 int8   
 12  판매건수_시차3                 int8   
 13  평균 판매가_시차1               float32
 14  평균 판매가_시차2               float32
 15  평균 판매가_시차3               float32
 16  상품ID별 평균 판매량_시차1         float32
 17  상품ID별 평균 판매량_시차2         float32
 18  상품ID별 평균 판매량_시차3         float32
 19  상품ID 도시별 평균 판매량_시차1      float32
 20  상품ID 도시별 평균 판매량_시차2      float32
 21  상

In [49]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['날짜ID'] < 33]
X_train = X_train.drop(['총 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['날짜ID'] == 33]
X_valid = X_valid.drop(['총 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['날짜ID'] == 34]
X_test = X_test.drop(['총 판매량'], axis=1)

# 훈련 데이터 (타깃 값)
y_train = all_data[all_data['날짜ID'] < 33]['총 판매량']
# 검증 데이터 (타깃 값)
y_valid = all_data[all_data['날짜ID'] == 33]['총 판매량']

# 가비지 컬렉션
del all_data
gc.collect();

## 모델 훈련 및 제출

In [50]:
import lightgbm as lgb

# lgb 하이퍼 파라미터
params = {'metric': 'rmse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise' : True,
          'random_state': 10}

cat_features = ['상점ID', '도시', '상품범주ID', '범주분류', '월']

# lgb 훈련 및 검증 데이터세트
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)
 
# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100)      

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 4034
[LightGBM] [Info] Number of data points in the train set: 9452298, number of used features: 30
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.297707
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.01226	valid_1's rmse: 0.98804
[200]	training's rmse: 0.91119	valid_1's rmse: 0.924923
[300]	training's rmse: 0.860293	valid_1's rmse: 0.900102
[400]	training's rmse: 0.832298	valid_1's rmse: 0.889985
[500]	training's rmse: 0.813863	valid_1's rmse: 0.886241
[600]	training's rmse: 0.800318	valid_1's rmse: 0.885718
[700]	training's rmse: 0.79032	valid_1's rmse: 0.885426
[800]	training's rmse: 0.782102	valid_1's rmse: 0.885614
Early stopping, best iteration is:
[749]	training's rmse: 0.786149	valid_1's rmse: 0.885292


In [51]:
preds = lgb_model.predict(X_test).clip(0,20)

submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)

In [52]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();